<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b> LSST Crowded Fields: LSST Galactic Bulge Data </b><br>
Select visits from LSSTCam that overlap the galactic bulge. <br> <br>

Contact author: Audrey Budlong <br>
Last verified to run: 10 February 2026 <br>

*[DP1 Reference](https://dp1.lsst.io/tutorials/notebook/301/notebook-301-1.html)* <br>
*[Querying Reference 1](https://github.com/lsst/tutorial-notebooks/blob/main/DP1/100_How_to_Use_RSP_Tools/104_Butler_data_access/104_3_Image_queries_with_the_butler.ipynb)* <br>
*[Querying Reference 2](https://rubin-obs.slack.com/archives/C08CANY4B6H/p1751305452604709)*

### Notebook Contents:
1. LSSTCam Coverage of Bulge Reference
2. Imports & Setup
4. Define Butler
5. Define Location at Bulge Center
6. Select Visits

### 1. LSSTCam Coverage of Bulge Reference

<img align="left" src = galacticBulge.jpeg width=1000 style="padding: 10px">

### 2. Imports & Setup

In [ ]:
from lsst.daf.butler import Butler, Timespan
import lsst.afw.display as afwDisplay
import lsst.sphgeom as sphgeom
import lsst.geom as geom
from astropy.time import Time
import matplotlib.pyplot as plt

afwDisplay.setDefaultBackend("firefly")

### 3. Define Butler

In [ ]:
collections = [
                "LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195",
                "skymaps",
            ]

instrument="LSSTCam"
skymap="lsst_cells_v1"
repo="dp2_prep"

butler = Butler(repo, instrument=instrument, collections=collections, skymap=skymap)
assert butler is not None

### 4. Define Location at Bulge Center

In [ ]:
ra = 300.00
dec = -20.00
radius = 2.5

In [ ]:
# region = sphgeom.Region.from_ivoa_pos("CIRCLE 53.076 -28.110 2.0")
region = sphgeom.Region.from_ivoa_pos(f"CIRCLE {ra} {dec} {radius}")

In [ ]:
point = geom.SpherePoint(ra*geom.degrees, dec*geom.degrees)
skymap = butler.get("skyMap", skymap="lsst_cells_v1")
tract = skymap.findTract(point).tract_id
patch = skymap.findTract(point).findPatch(point).getSequentialIndex()
print(tract, patch)

In [ ]:
band = 'r'

In [ ]:
image_types = ['deep_coadd', 'visit_image',
               'difference_image', 'template_coadd']
for itype in image_types:
    print('')
    print(itype)
    print(butler.get_dataset_type(itype))
    print('required: ', butler.get_dataset_type(itype).dimensions.required)

### 5. Select Visits

In [ ]:
query = f"band.name = '{band}' AND \
visit_detector_region.region OVERLAPS POINT({ra}, {dec})"

dataset_refs = butler.query_datasets("visit_image",
                                     where=query)

In [ ]:
len(dataset_refs)

In [ ]:
visitList = []
for dataref in dataset_refs:
    visitList.append(dataref.dataId["visit"])

In [ ]:
visitList